#Задание
Пакет SURPRISE:

* используйте данные MovieLens 1M,

* можно использовать любые модели из пакета,

* получите RMSE на тестовом сете 0,87 и ниже.

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens1M.zip

--2024-02-01 05:55:28--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 173.194.216.138, 173.194.216.139, 173.194.216.102, ...
Connecting to drive.google.com (drive.google.com)|173.194.216.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-02-01 05:55:28--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.196.132, 2607:f8b0:400c:c36::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.196.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens1M.zip’

MovieLens1M.zip     100%[===================>] 827.83K  --.-KB/s    in 0.006s  

2024-02-01 05:55:29 (143 MB

In [ ]:
!unzip MovieLens1M.zip

Archive:  MovieLens1M.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162670 sha256=709add4072f2ac15a4dd6b9355573d564420d73dba14f4eb7681fef7e627227f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

## SVD

In [ ]:
# Загрузка данных
data = Dataset.load_builtin('ml-1m')

Dataset ml-1m could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
# Инициализация и обучение модели SVD
algo = SVD()
algo.fit(trainset)

In [ ]:
# Получение прогнозов на тестовом наборе
predictions = algo.test(testset)
predictions[:3]

[Prediction(uid='4378', iid='2764', r_ui=4.0, est=3.501655002427066, details={'was_impossible': False}),
 Prediction(uid='1060', iid='1573', r_ui=5.0, est=3.8570428654428155, details={'was_impossible': False}),
 Prediction(uid='4510', iid='2001', r_ui=4.0, est=3.0513103632007224, details={'was_impossible': False})]

In [ ]:
# Оценка точности прогнозов
accuracy.rmse(predictions)

RMSE: 0.8736


0.8736476030592082

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
# Проведение кросс-валидации
cv_results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8733  0.8719  0.8760  0.8760  0.8745  0.8743  0.0016  
Fit time          16.72   17.33   17.65   17.06   17.17   17.18   0.31    
Test time         2.54    2.58    2.44    3.90    3.21    2.93    0.55    


## NMF

In [ ]:
# Инициализация и обучение модели NMF
algo2 = NMF(n_epochs=300)
algo2.fit(trainset)

In [ ]:
# Получение прогнозов на тестовом наборе
predictions2 = algo2.test(testset)
predictions2[:3]

[Prediction(uid='4378', iid='2764', r_ui=4.0, est=3.1160158973807754, details={'was_impossible': False}),
 Prediction(uid='1060', iid='1573', r_ui=5.0, est=3.732881588991191, details={'was_impossible': False}),
 Prediction(uid='4510', iid='2001', r_ui=4.0, est=2.821732171254384, details={'was_impossible': False})]

In [ ]:
# Оценка точности прогнозов
accuracy.rmse(predictions2)

RMSE: 0.8608


0.8608461923563888

In [ ]:
# Проведение кросс-валидации
cv_results2 = cross_validate(algo2, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8600  0.8578  0.8625  0.8622  0.8630  0.8611  0.0019  
Fit time          138.07  126.76  139.05  131.36  139.19  134.89  4.99    
Test time         2.96    2.02    2.43    2.11    1.89    2.28    0.38    
